Входная коллекция данных состоит из двух частей:
1. заголовки новостных сообщений, входящих в топ-25 по популярности, опубликованных на  Reddit WorldNews Channel
2. бинаризованный промышленный индекса Доу — Джонса DJIA (0, если значение индекса за день понизилось, 1 – если осталось на прежнем уровне или выросло). 

Данные собраны за период с 2008-08-08 до 2016-07-01.

Данные представлены в файле Combined_News_DJIA.csv. Первая колонка – дата, вторая – бинаризованный промышленный индекса Доу — Джонса, оставшиеся 25 – заголовки новостей. 

### Часть 1 [3 балла] Предварительная обработка текстов
Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 
Ответьте на следующие вопросы:
1. Есть ли корреляция между средней длинной текста за день и DJIA?
2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.
3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?
4. О каких кризисах (crisis) пишут статьи?


In [6]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
import numpy as np
import pymorphy2 as pm

In [7]:
df = pd.read_csv('Combined_News_DJIA.csv')
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
morph = pm.MorphAnalyzer()
def set_clean(s):
    try:
        clean_line = re.sub('[\W\d_-]+', ' ', s.lower().strip())
        return re.sub(' +', ' ', clean_line)
    except AttributeError:
        print ("this was a series")
print(set_clean((df['Top1'][1])))

b why wont america and nato help us if they wont help us now why did we help them in iraq 


In [5]:
df_small = df.drop(['Date', 'Label'], axis=1)
#df_small

In [6]:
df_small.apply(set_clean)

this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series
this was a series


Top1     None
Top2     None
Top3     None
Top4     None
Top5     None
Top6     None
Top7     None
Top8     None
Top9     None
Top10    None
Top11    None
Top12    None
Top13    None
Top14    None
Top15    None
Top16    None
Top17    None
Top18    None
Top19    None
Top20    None
Top21    None
Top22    None
Top23    None
Top24    None
Top25    None
dtype: object

Считаем среднюю длину текста за день и ее отношение к индексу

In [7]:
df_clean = pd.DataFrame(df)

In [8]:
def meaning(columns, index):
    den = len(columns)
    acc = 0
    for i in range(den):
        try:
            acc+=len(columns[i][index])
        except TypeError:
            den-=1
    return acc/den
def meanings(columns):
    l = len(columns[0])
    return [{'dj': df['Label'][i], 'meaning': meaning(columns, i)} for i in range(l)]
cols = [df["Top{0}".format(i)] for i in range(1, 26)]
cols_new = meanings(cols)

In [16]:
type(df_clean)

pandas.core.frame.DataFrame

In [10]:
cols_df = pd.DataFrame(cols_new)


In [11]:
cols_sort = cols_df.sort_values(by='meaning', ascending=False)
cols_sort

,dj,meaning
276,1,176.00
1254,1,171.40
604,1,169.48
544,1,165.92
908,0,163.96
600,0,160.96
547,1,158.84
312,0,157.56
1412,1,155.16
1270,0,154.76


Упоминания Обамы, США и индекс??

Кого чаще упоминают: Россию или ИГИЛ?

In [19]:
def word_count(columns, word):
    word_list = []
    for col in columns:
        for v in col:
            if (type(v)==pd.core.frame.DataFrame and re.search("{word}".format, v)):
                pattern = "((\\w+\\W+)\\b|\w+) word"
                try:
                    vv = re.search(pattern, v).group(0)
                    word_list.append(vv)
                except AttributeError:
                    continue
    return print(len(word_list))

word_count(cols, 'Putin')
word_count(cols, 'Russia')
word_count(cols, 'ISIS')

0
0
0


In [185]:
def Russia(columns):
    Russia_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('Russia', v)):
                pattern = "((\\w+\\W+)\\b|\w+)Russia"
                try:
                    vv = re.search(pattern, v).group(0)
                    Russia_list.append(vv)
                except AttributeError:
                        continue
    return print(len(Russia_list))

def Russia1(columns):
    Russian_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('Russian', v)):
                pattern = "((\\w+\\W+)\\b|\w+)Russian"
                try:
                    vv = re.search(pattern, v).group(0)
                    Russian_list.append(vv)
                except AttributeError:
                    continue
    return print(len(Russian_list))

def Putin(columns):
    Putin_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('Putin', v)):
                pattern = "((\\w+\\W+)\\b|\w+)Putin"
                try:
                    vv = re.search(pattern, v).group(0)
                    Putin_list.append(vv)
                except AttributeError:
                    continue
    return print(len(Putin_list))
Russia(cols)
Russia1(cols)
Putin(cols)


1957
923
371


In [186]:
def ISIS(columns):
    ISIS_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('ISIS', v)):
                pattern = "((\\w+\\W+)\\b|\w+)ISIS"
                try:
                    vv = re.search(pattern, v).group(0)
                    ISIS_list.append(vv)
                except AttributeError:
                        continue
    return print(len(ISIS_list))

def IS(columns):
    IS_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('IS', v)):
                pattern = "((\\w+\\W+)\\b|\w+)IS"
                try:
                    vv = re.search(pattern, v).group(0)
                    IS_list.append(vv)
                except AttributeError:
                    continue
    return print(len(IS_list))

ISIS(cols)
IS(cols)


389
858


О каких кризисах пишут в статьях?

In [74]:
def crisis(columns):
    crisis_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('crisis', v)):
                crisis_list.append(v)
    return crisis_list

In [113]:
# print((crisis(cols)))
c = crisis(cols)[0]
re.search("(\\w+\\W+){,2}\\bcrisis", c).group(0)

'and humanitarian crisis'

In [118]:
def crisis(columns):
    crisis_list = []
    for col in columns:
        for v in col:
            if (type(v)==str and re.search('crisis', v)):
                pattern = "((\\w+\\W+){,2}\\b|\w+)crisis"
                try:
                    vv = re.search(pattern, v).group(0)
                    crisis_list.append(vv)
                except AttributeError:
                    print("\n%%%%%  ")
                    print(v)
                    print(re.search(pattern, v))
    return crisis_list
crisis(cols)

['and humanitarian crisis',
 'puts current crisis',
 'the difficult crisis',
 'a humanitarian crisis',
 'the banking crisis',
 'the nuclear crisis',
 'the nuclear crisis',
 'the financial crisis',
 'the financial crisis',
 'coral bleaching crisis',
 'the humanitarian crisis',
 'Africa housing crisis',
 'future health crisis',
 'Generation  -- The crisis',
 'the climate crisis',
 'the financial crisis',
 'Migrant crisis',
 'The economic crisis',
 'drinking water crisis',
 'the Google crisis',
 'raise Fukushima crisis',
 "Japan's crisis",
 'from Iceland crisis',
 'next financial crisis',
 'of food crisis',
 'Gaza crisis',
 'sperm count crisis',
 'North Korea crisis',
 'the financial crisis',
 'global financial crisis',
 'Migration crisis',
 'S. financial crisis',
 's cholera crisis',
 'into total crisis',
 'redshirts as crisis',
 'the financial crisis',
 'Korean hunger crisis',
 'a financial crisis',
 'brink of crisis',
 'the Ukraine crisis',
 'the financial crisis',
 'amid russia crisis

### Часть 2 [5 баллов] Классификация

Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится DJIA. То есть, метки класса (y) заданы DJIA, признаки (X) требуется извлечь из текстов.

Обучающее и тестовое множество строится так: данные до начала 2015 года используются для обучения, данные с 2015 года и позже – для тестировани.

Подсказка:

In [1]:
import pandas.io.date_converters as conv

In [8]:
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']

In [9]:
type(train)

pandas.core.frame.DataFrame

In [10]:
train_dict = train.to_dict()

In [12]:
test_dict = test.to_dict()

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train)
X_train_counts.shape

(27, 27)

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(27, 27)

In [21]:
print(train)

            Date  Label                                               Top1  \
0     2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
1     2008-08-11      1  b'Why wont America and Nato help us? If they w...   
2     2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
3     2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
4     2008-08-14      1  b'All the experts admit that we should legalis...   
5     2008-08-15      1  b"Mom of missing gay man: Too bad he's not a 2...   
6     2008-08-18      0  b'In an Afghan prison, the majority of female ...   
7     2008-08-19      0  b"Man arrested and locked up for five hours af...   
8     2008-08-20      1  b'Two elderly Chinese women have been sentence...   
9     2008-08-21      1  b"British resident held in Guantanamo Bay wins...   
10    2008-08-22      1  b'Syria says its ready to put a Russian missil...   
11    2008-08-25      0  b"N Korea's Kim died in 2003; replaced 

[1611 rows x 27 columns]


In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, test)

ValueError: bad input shape (378, 27)

In [25]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])
text_clf = text_clf.fit(train, test)

ValueError: bad input shape (378, 27)

Используйте любой известный вам алгоритм классификации текстов для того,  Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure  для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. 

Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации. 